In [9]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt

# import torch.utils.data as data
from itertools import combinations
from scipy.stats import linregress
from sklearn.preprocessing import StandardScaler

signals = [
  loadmat('mat_signals/hr_data.mat')['hr_data'],
  # loadmat('mat_signals/hbo_data.mat')['hbo_data'],
  loadmat('mat_signals/eda_data.mat')['eda_data'],
  loadmat('mat_signals/hrv_data.mat')['hrv_data'],
  loadmat('mat_signals/temp_data.mat')['temp_data'],
]
signal_names = ['hr', 'eda', 'hrv', 'temp']

#signals are 3d arrays where first dim is subj, second dim signal amplitude, and third dim is time
column_names = [
    'labels',
    'hr_mean', 'hr_std', 'hr_slp', 'hr_skew', 'hr_kurt',
    'hbo_mean', 'hbo_std', 'hbo_slp', 'hbo_skew', 'hbo_kurt',
    'eda_mean', 'eda_std', 'eda_slp', 'eda_skew', 'eda_kurt',
    'hrv_mean', 'hrv_std', 'hrv_slp', 'hrv_skew', 'hrv_kurt',
    'temp_mean', 'temp_std', 'temp_slp', 'temp_skew', 'temp_kurt'
]

In [10]:
def label_intensity_by_type(type, curr_signal, signal_name, star_subj, whole_df):
    """Groups the signals by task workload (none, low, med, high) of the specified type (cog,phys,tot), 
    adds labels and returns a dataframe of all the signals with numbered subjects"""
    signal_df = pd.DataFrame(curr_signal)
    signal_df = signal_df.transpose()
    for i in range(0,29):
      curr_subj = signal_df.iloc[i,0]
      if type == "cog":
        none = curr_subj[:, ::4]
        low = curr_subj[:, 1::4]
        med = curr_subj[:, 2::4]
        high = curr_subj[:, 3::4]
      elif type == "phys":
        none = np.hstack((curr_subj[:, 0:4], curr_subj[:, 16:20]))
        low = np.hstack((curr_subj[:, 4:8], curr_subj[:, 20:24]))
        med = np.hstack((curr_subj[:, 8:12], curr_subj[:, 24:28]))
        high = np.hstack((curr_subj[:, 12:16], curr_subj[:, 28:32]))
      elif type == "tot":
        none = np.hstack((curr_subj[:, 0:1], curr_subj[:, 1:2], curr_subj[:, 3:4], curr_subj[:, 4:5], curr_subj[:, 5:6], curr_subj[:, 8:9], curr_subj[:, 16:17], curr_subj[:, 17:18], curr_subj[:, 19:20], curr_subj[:, 20:21], curr_subj[:, 21:22], curr_subj[:, 24:25]))
        low = np.hstack((curr_subj[:, 3:4], curr_subj[:, 6:7], curr_subj[:, 9:10], curr_subj[:, 12:13], curr_subj[:, 19:20], curr_subj[:, 22:23], curr_subj[:, 25:26], curr_subj[:, 28:29]))
        med = np.hstack((curr_subj[:, 7:8], curr_subj[:, 10:11], curr_subj[:, 13:14], curr_subj[:, 23:24], curr_subj[:, 26:27], curr_subj[:, 29:30]))
        high = np.hstack((curr_subj[:, 11:12], curr_subj[:, 14:15], curr_subj[:, 15:16],curr_subj[:, 27:28], curr_subj[:, 30:31], curr_subj[:, 31:32] ))
      
      labeled_df = pd.DataFrame(index=range(32), columns=["label",signal_name])
      labeled_df.insert(0, 'subj', i+1)
      for j in range(8):
        row = none[:,j]  
        labeled_df.at[j, signal_name] = row
        labeled_df.at[j, "label"] = 0
      for j in range(8):
        row = low[:,j]  
        labeled_df.at[j+8, signal_name] = row
        labeled_df.at[j+8, "label"] = 1
      for j in range(8):
        row = med[:,j]  
        labeled_df.at[j+16, signal_name] = row
        labeled_df.at[j+16, "label"] = 2
      for j in range(8):
        row = high[:,j]  
        labeled_df.at[j+24, signal_name] = row
        labeled_df.at[j+24, "label"] = 3

      if signal_name == "hr":
        whole_df = pd.concat([whole_df, labeled_df], axis=0, ignore_index=True)
      else:
        if signal_name not in whole_df.columns:
          whole_df.insert(len(whole_df.columns), signal_name, labeled_df[signal_name])
        else:
          whole_df.loc[(32*i):((32*i-1)+32), signal_name] = labeled_df[signal_name].values
    return whole_df

def extract_features(windowed_df, signal_names):
    """Extracts mean and standard deviation features from the input signal."""
    features_list = ["mean", "std", "slope", "skewness", "kurtosis"]
    #make new df to hold features. copy labels and subjs
    features_df = windowed_df[['subj', 'label']].copy()
    
    # iterate through each signal (columns of windowed_df)
    for col in range(len(signal_names)):
      # iterate through each row in the columns
      a=1
      for feat in features_list:
        signal = signal_names[col]
        features_df[f"{signal}_{feat}"] = np.nan
      for row in range(windowed_df.shape[0]):
        cell = windowed_df.iloc[row, col+2]  
        #if array is all nans - remove entire row
        if np.isnan(cell).all():
          aa = 1 #TODO remove whole row 
        else:   
          #calculate features
          for f in range(len(features_list)):
            feature_val = calc_feature(features_list[f], cell)
            features_df.iloc[row, (f+(5*col)+2)] = feature_val

    return features_df

def calc_feature(feature, matrix):
    """Calculates the specified feature (mean or std) for the given matrix and appends it to the feature list."""
    if feature == "mean":
      result = np.nanmean(matrix, axis=0)
    elif feature == "std":
      result = np.nanstd(matrix, axis=0)
    elif feature == "slope":
      notnan = matrix[~np.isnan(matrix)]
      num_samples = len(notnan)
      x = np.arange(num_samples)  
      result, intercept, _, _, _ = linregress(x, notnan.T)
    elif feature == "skewness":
      result = np.apply_along_axis(lambda x: pd.Series(x).skew(), axis=0, arr=matrix)
    elif feature == "kurtosis":
      result = np.apply_along_axis(lambda x: pd.Series(x).kurtosis(), axis=0, arr=matrix)
    else:
      raise ValueError("Unsupported feature type")
    return result

def window_star_subj_signals(whole_df, signal_names, star_subj, windows):
  """Splits signals from star_subj into X windows to increase sample size."""
  star_data = whole_df.loc[whole_df['subj'] == star_subj]
  star_data.reset_index(drop=True, inplace=True)
  new_star = pd.DataFrame(np.nan, index=range(32*windows),columns=star_data.columns)
  new_star = new_star.astype('object')
  new_star['subj'] = star_subj
  for r in range(star_data.shape[0]):
    for c in range(2,star_data.shape[1]):
      curr = star_data.iloc[r,c] 
      if not np.isnan(curr).all():
        curr = curr.reshape(-1, 1).T
        curr = curr[~np.isnan(curr)]
        split = np.array_split(curr, round(len(curr)/windows))  
        split = [row for row in split if len(row) == 4]
        split = np.array(split)
        for w in range(windows):
          new_star.at[(r*windows)+w,signal_names[c-2]] = split[:,w]
          new_star.at[(r*windows)+w,'label'] = star_data.loc[r,'label']


  windowed_df = pd.concat([whole_df, new_star], axis=0, ignore_index=True)

  return windowed_df

# -------------------------
#      Runner section
# -------------------------

# -------Variables---------
# List of features to consider
subset_features = list(range(26)) #for all feaures use  list(range(26))  , or [0,1, 2,4,12]  etc. (must include 0 and 1 for labels and subj)
c_reg = 1.5 # regularization parameter (arbitrarily large makes this a hard margin SVM)
# star_subj = np.random.randint(9, 30)
star_subj = 14 #can't be subj 1,2,3 because missing HRV data, 4/5/7 and kinda 8 are missing whole phys levels
star_weight = 30
windows = 4
groupings = ["phys"] # iterates all code across the different groupings. For all 3 use:  ["phys", "cog", "tot"]


# ----------Main---------------
for group_by in groupings:
  # ----------Extract features, group signals, preprocess data---------------
  whole_df = pd.DataFrame()
  for i, curr_signal in enumerate(signals):
    whole_df = label_intensity_by_type(group_by, curr_signal, signal_names[i], star_subj, whole_df)  

  windowed_df = window_star_subj_signals(whole_df, signal_names, star_subj, windows)
  features_df = extract_features(windowed_df, signal_names)
    
  # Drop rows with missing values 
  missing_data = features_df[features_df.isnull().any(axis=1)].copy()
  missing_data['row_index'] = missing_data.index
  final_df = features_df.dropna().reset_index(drop=True)

  # Apply z-score standardization
  scaler = StandardScaler()
  final_df.iloc[:, 2:] = scaler.fit_transform(final_df.iloc[:, 2:])


/opt/miniconda3/envs/thesisenv/lib/python3.9/site-packages/scipy/stats/_stats_mstats_common.py:182: RuntimeWarning: invalid value encountered in scalar divide
  slope = ssxym / ssxm
/opt/miniconda3/envs/thesisenv/lib/python3.9/site-packages/scipy/stats/_stats_mstats_common.py:196: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/opt/miniconda3/envs/thesisenv/lib/python3.9/site-packages/scipy/stats/_stats_mstats_common.py:199: RuntimeWarning: invalid value encountered in scalar divide
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)
/opt/miniconda3/envs/thesisenv/lib/python3.9/site-packages/scipy/stats/_stats_mstats_common.py:182: RuntimeWarning: invalid value encountered in scalar divide
  slope = ssxym / ssxm
/opt/miniconda3/envs/thesisenv/lib/python3.9/site-packages/scipy/stats/_stats_mstats_common.py:196: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/